## Practice Problem 1: AB Mower

In [9]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('SolveIntegerProblem',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


material_price = 3000 
holding_price = 100 
stockout_cost = 600 
hiring_price = 5000 
firing_price = 100000 
labour_hours_per_item = 10
regular_worker_price_month = 24000 
overtime_price_hour = 220 
outsourcing_price_item = 5500

hrs_per_worker_per_month = 160
max_overtime = 20
i0 = 500
iend = 500
initial_backlog = 0
ending_backlog = 0
workforce0 = 200
min_end_workforce = 170
max_end_workforce = 230

selling_price = 6000



demand = [0, 2800, 2800, 2900, 4000, 4500, 
          4200, 4100, 3300, 3100, 2600, 2400, 2800]
total_demand = sum(demand)
n_t = len(demand)

## variables

inventory = [solver.IntVar(0, total_demand, 'inv_{t}'.format(t=t))\
            for t in range(n_t)]

workforce = [solver.IntVar(int(0.5*min_end_workforce),
                           int(2*max_end_workforce),
                           'workforce_{t}'.format(t=t))\
             for t in range(n_t)]

#(hours_worked)

employees_hired = [solver.IntVar(0,
                                 max_end_workforce,
                                 'hiring_{t}'.format(t=t))\
                   for t in range(n_t)]

employees_fired = employees_hired = [solver.IntVar(0,
                                 max_end_workforce,
                                 'firing_{t}'.format(t=t))\
                   for t in range(n_t)]

overtime_hours = [solver.IntVar(0,
                                 demand[t]*labour_hours_per_item,
                                 'overtime_{t}'.format(t=t))\
                   for t in range(n_t)]

backlog = [solver.IntVar(0, total_demand, 'backlog_{t}'.format(t=t))\
            for t in range(n_t)]

stockout_boolean = [solver.BoolVar('stockout_{t}'.format(t=t)) for t in range(n_t)]

native_production = [solver.IntVar(0, total_demand, 'native_prod_{t}'.format(t=t))\
            for t in range(n_t)]

outsourced_production = [solver.IntVar(0, total_demand, 'outsourced_prod_{t}'.format(t=t))\
            for t in range(n_t)]


### costs

regular_labour_costs = solver.Sum([workforce[t]*regular_worker_price_month for t in range(n_t)])

overtime_labour_costs = solver.Sum([overtime_hours[t]*overtime_price_hour for t in range(n_t)])

hiring_costs = solver.Sum([employees_hired[t]*hiring_price for t in range(n_t)])

firing_costs = solver.Sum([employees_fired[t]*firing_price for t in range(n_t)])

inventory_costs = solver.Sum([inventory[t]*holding_price for t in range(n_t)])

stockout_costs = solver.Sum([stockout_boolean[t]*stockout_cost for t in range(n_t)])

material_costs = solver.Sum([native_production[t]*material_price for t in range(n_t)])

outsourcing_costs = solver.Sum([outsourced_production[t]*outsourcing_price_item for t in range(n_t)])


## constraints

## initialisations



# production + backlog + iventory [-1] = demand + backlog[-1] + iventory
for t in range(1, n_t):
    fin = solver.Sum([native_production[t], backlog[t], inventory[t-1]])
    fout = solver.Sum([demand[t], backlog[t], inventory[t]])
    solver.Add(fin==fout)


# link stockoutboolean to backlogck
for i in t range(n_t):
    solver.Add(backlog[t] -M*stockout_boolean[t]<=0)